In [4]:
import arcpy

arcpy.env.overwriteOutput = True

In [5]:
output_folder = "C:/EsriTraining/"
output_gdb = "AlphaCentauryB.gdb"

out_gdb = output_folder + output_gdb
arcpy.env.workspace = out_gdb

In [6]:
if not arcpy.Exists(out_gdb):
    arcpy.CreateFileGDB_management(output_folder, output_gdb)

In [129]:
rasters = arcpy.ListRasters()
print(rasters)

['Temperatures', 'Precipitations', 'Radiations']


In [134]:
import time

def Calculation (tMin, tCoef, rCoef, pCoef):
    
    timed = time.time()
    arcpy.management.Delete('tNorm')
    arcpy.management.Delete('pNorm')
    arcpy.management.Delete('rNorm')
    arcpy.management.Delete('Final')
    arcpy.management.Delete('Final80')

    
    
    
    
    tRast = arcpy.Raster("Temperatures")
    tBottom = tMin
    tMid = 0
    tTop = tRast.maximum

    tNormD = (tRast - tBottom) / (tMid - tBottom) * 60
    tNormU = (tRast - tMid) / (tTop - tMid) * 40
    tNormS = tNormD + tNormU

    tNormR = arcpy.sa.Con(tNormS, 100, tNormS, "VALUE > 100")
    tNormR2 = arcpy.sa.Con(tNormR, 0, tNormR, "VALUE <0")

    tNormR2.save("tNorm")



    
    
    pRast = arcpy.Raster("Precipitations")
    pBottom = pRast.minimum
    pTop = pRast.maximum

    pNorm = (pRast - pBottom) / (pTop - pBottom) * 100

    pNorm.save("pNorm")


    
    
    
    rRast = arcpy.Raster("Radiations")
    rBottom = rRast.minimum
    rTop = rRast.maximum
    rMid = (rBottom + rTop)/2

    rNorm = ((rRast - rBottom)/(rTop - rBottom)*100)-50

    rNormR = arcpy.sa.Con(rNorm, (rNorm*(-1)+50)-1000, rNorm, "VALUE >=25")
    rNormR2 = arcpy.sa.Con(rNormR, (rNormR*(-1)*3+100)-500, rNormR, "VALUE >=0")
    rNormR3 = arcpy.sa.Con(rNormR2, ((rNormR2*3)+100)-500, rNormR2, "VALUE >=-25")
    rNormR4 = arcpy.sa.Con(rNormR3, (rNormR3 +50)-1000, rNormR3, "VALUE >=-50")
    rNormR5 = arcpy.sa.Con(rNormR4, rNormR4+500, rNormR4, "VALUE >-500")
    rNormR6 = arcpy.sa.Con(rNormR5, rNormR5+1000, rNormR5, "VALUE <-500")
    rNormR6.save("rNorm")

    
    
    
    finalmap = (tNormR2*tCoef + pNorm*pCoef + rNormR6*rCoef)
    finalmap.save("Final")

    finalmapNorm = arcpy.sa.Con(finalmap, finalmap/4, finalmap, "VALUE < 80")
    finalmapNorm.save("Final80")

    
    
    

    ftime = time.time()
    temps = ftime - timed
    print("Calculation duration : "+str(temps))
    
    
    

def Verification():
    tRast = arcpy.Raster("Temperatures")
    tmin = tRast.minimum
    tmax = tRast.maximum
    tMin = input("Enter the value for the minimum temperature. Between "+str(tmin)+" and "+str(tmax)+"   : ")
    tMin = float(tMin)
    while not(tMin>tmin and tMin<tmax):
        tMin = input("Incorrect value. Between "+str(tmin)+" and "+str(tmax)+" : ")
        tMin = float(tMin)
    print("Enter the weight of Temperature, Precipitations, Radiations. These numbers must be beetween 0 and 1 with their sum equals to 1")
    tCoef = input("Enter the weight of Temperature : ")
    pCoef = input("Enter the weight of Precipitations : ")
    rCoef = input("Enter the weight of Radiations : ")
    tCoef = float(tCoef)
    pCoef = float(pCoef)
    rCoef = float(rCoef)
    sCoef = tCoef+pCoef+rCoef
    while not(sCoef==1 or (sCoef>0.99 and sCoef<=1)):
        print("Sum of weight equals : "+str(sCoef)+" it must be 1")
        tCoef = input("Enter the weight of Temperature : ")
        pCoef = input("Enter the weight of Precipitations : ")
        rCoef = input("Enter the weight of Radiations : ")
        tCoef = float(tCoef)
        pCoef = float(pCoef)
        rCoef = float(rCoef)
        sCoef = tCoef+pCoef+rCoef
    Calculation(tMin, tCoef, pCoef, rCoef)
        
        
Verification() #Default Values :   -20   :   0.5   :   0.2   :   0.3

Enter the value for the minimum temperature. Between -64.51473999023438 and 2.6605117321014404   : -30
Enter the weight of Temperature, Precipitations, Radiations. These numbers must be beetween 0 and 1 with their sum equals to 1
Enter the weight of Temperature : 0.6
Enter the weight of Precipitations : 0.1
Enter the weight of Radiations : 0.3
Calculation duration : 46.73843026161194
